In [36]:
import sys
print(sys.executable)
print(sys. version)

C:\Users\carlo\anaconda3\python.exe
3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:20:11) [MSC v.1938 64 bit (AMD64)]


In [5]:
from rcsbsearchapi.search import TextQuery
from rcsbsearchapi import rcsb_attributes as attrs

C:\Users\carlo\AppData\Local\Temp\ipykernel_13000\1820887722.py:1: DeprecationWarning: Please migrate to the use of our new and improved package, rcsb-api (https://rcsbapi.readthedocs.io/en/latest/),
    which contains all the same functionalities as rcsbsearchapi and more! New features will only be added to the new rcsb-api package.
    For more details, see https://github.com/rcsb/py-rcsbsearchapi/issues/51.
  from rcsbsearchapi.search import TextQuery


In [6]:
ECnumber = "7.2.2.13"    

q1 = attrs.rcsb_polymer_entity.rcsb_ec_lineage.id == ECnumber  # looking for Na/Kaptase
q3 = TextQuery('BUF') 
query = q1 & q3              

results = list(query())
print(results)

['4RES', '7D94', '7DDL']


In [8]:
pdb_id = results[2].lower() ## Get the PDB ID from the list and convert it to lowercase
print(pdb_id)

7ddl


In [11]:
import os # for making directories
import requests

# make a directory for pdb files
protein_directory = "protein_structures"

## fill in function to make directories
os.makedirs(protein_directory, exist_ok=True)

pdb_request = requests.get(f"https://files.rcsb.org/download/7ddl.pdb")
pdb_request.status_code

200

In [13]:
with open(f"{protein_directory}/{pdb_id}.pdb", "w+") as f:
    ## fill in write command
    f.write(pdb_request.text)

## Visualizing the protein strucure

In [16]:
import MDAnalysis as mda

# Load into MDA universe
u = mda.Universe(f'{protein_directory}/{pdb_id}.pdb')
u

<Universe with 21312 atoms>

In [17]:
import nglview as nv
view=nv.show_mdanalysis(u)
view
## create and show an NGLView from an MDAnalysis universe

NGLWidget()

In [18]:
# Select protein atoms
protein = u.select_atoms("protein")
ligandB = u.select_atoms("resname BUF")
water = u.select_atoms('resname HOH')## fill in selection for water.
print("ligandB:", len(ligandB))
print("water:", len(water))

ligandB: 56
water: 10


In [19]:
view = nv.show_mdanalysis(protein)
view.clear_representations()
view.add_representation('cartoon', ColorScheme='resname')
lig_viewB = view.add_component(ligandB)
lig_viewB.add_representation("ball+stick")
water_view = view.add_component(water)
water_view.add_representation("spacefill")
view

NGLWidget()

In [24]:
# Write protein to new PDB file
protein.write(f"{protein_directory}/protein_{pdb_id}.pdb")

## Fixing the protein structure

In [27]:
!python -m pdb2pqr --pdb-output=protein_structures/protein_7ddl.pdb --pH=7.4 protein_structures/protein_7ddl.pdb protein_structures/protein_7ddl.pqr --whitespace

INFO:PDB2PQR v3.6.1: biomolecular structure conversion software.
INFO:Please cite:  Jurrus E, et al.  Improvements to the APBS biomolecular solvation software suite.  Protein Sci 27 112-128 (2018).
INFO:Please cite:  Dolinsky TJ, et al.  PDB2PQR: expanding and upgrading automated preparation of biomolecular structures for molecular simulations. Nucleic Acids Res 35 W522-W525 (2007).
INFO:Checking and transforming input arguments.
INFO:Loading topology files.
INFO:Loading molecule: protein_structures/protein_7ddl.pdb
ERROR:Error parsing line: invalid literal for int() with base 10: ''
ERROR:<REMARK     2>
ERROR:Truncating remaining errors for record type:REMARK

ERROR:['REMARK']
INFO:Setting up molecule.
INFO:Created biomolecule object with 2630 residues and 20666 atoms.
INFO:Setting termini states for biomolecule chains.
INFO:Loading forcefield.
INFO:Loading hydrogen topology definitions.
INFO:Attempting to repair 2 missing atoms in biomolecule.
INFO:Added atom OXT to residue LYS E 48 

## Saving a protein PDBQT File

In [29]:
# make a directory for pdb files
pdbqt_directory = 'pdbqt'## fill in the name of the directory to write PDBQT files to
os.makedirs(pdbqt_directory, exist_ok=True)

u = mda.Universe(f"{protein_directory}/protein_{pdb_id}.pqr")
u.atoms.write(f"{pdbqt_directory}/{pdb_id}.pdbqt")

C:\Users\carlo\anaconda3\Lib\site-packages\MDAnalysis\coordinates\PDBQT.py:305: UserWarning: Supplied AtomGroup was missing the following attributes: altLocs, occupancies, tempfactors. These will be written with default values. 
  warnings.warn(


In [30]:
# Read in the just-written PDBQT file, replace text, and write back
with open(f"{pdbqt_directory}/{pdb_id}.pdbqt", 'r') as file:
    file_content = file.read()

# Replace 'TITLE' and 'CRYST1' with 'REMARK'
file_content = file_content.replace('TITLE', 'REMARK').replace('CRYST1', 'REMARK')

# Write the modified content back to the file
with open(f"{pdbqt_directory}/{pdb_id}.pdbqt", 'w') as file:
    file.write(file_content)

In [38]:
!where mk_prepare_ligand.py  # Windows

C:\Users\carlo\anaconda3\Scripts\mk_prepare_ligand.py


INFORMACIÃ“N: no se pudo encontrar"#".
INFORMACIÃ“N: no se pudo encontrar"Windows".


In [46]:
!python C:/Users/carlo/anaconda3/Scripts/mk_prepare_ligand.py -i ligands/resBUF.sdf -o pdbqt/BUF.pdbqt

Traceback (most recent call last):
  File "C:\Users\carlo\anaconda3\Scripts\mk_prepare_ligand.py", line 14, in <module>
    from meeko import MoleculePreparation
  File "C:\Users\carlo\anaconda3\Lib\site-packages\meeko\__init__.py", line 32, in <module>
    from .rdkit_mol_create import RDKitMolCreate
  File "C:\Users\carlo\anaconda3\Lib\site-packages\meeko\rdkit_mol_create.py", line 11, in <module>
    from rdkit.six import StringIO
ModuleNotFoundError: No module named 'rdkit.six'


In [48]:
!python C:/Users/carlo/anaconda3/Scripts/mk_prepare_ligand.py -i ligands/resBUF.sdf -o pdbqt/BUF.pdbqt > out.txt 2>&1
!type out.txt


Traceback (most recent call last):
  File "C:\Users\carlo\anaconda3\Scripts\mk_prepare_ligand.py", line 14, in <module>
    from meeko import MoleculePreparation
  File "C:\Users\carlo\anaconda3\Lib\site-packages\meeko\__init__.py", line 32, in <module>
    from .rdkit_mol_create import RDKitMolCreate
  File "C:\Users\carlo\anaconda3\Lib\site-packages\meeko\rdkit_mol_create.py", line 11, in <module>
    from rdkit.six import StringIO
ModuleNotFoundError: No module named 'rdkit.six'


In [1]:
# Use meeko to prepare small molecules - using meeko helps us visualize them later.
! mk_prepare_ligand.py -i ligands/resBUF.sdf -o pdbqt/BUF.pdbqt
# Fill in the command for converting the N modified ligand.